In [1]:
import torch
from build_dataloader import get_train_loader, get_test_loader
from custom_resnet import CustomRes
from torchsummary import summary
from trainer import train, test
from torch.optim import SGD

In [2]:
BATCHSIZE=128
train_dataloader = get_train_loader(data_dir='./data', 
                                    train=True,
                                    download=True,
                                    shuffle=True,
                                    batch_size=BATCHSIZE
                                    )

test_dataloader = get_test_loader(data_dir='./data', 
                                    train=False,
                                    download=True,
                                    shuffle=True,
                                    batch_size=BATCHSIZE
                                    )

Files already downloaded and verified
Files already downloaded and verified


In [3]:
model = CustomRes()
summary(model, input_size=(3,32,32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,728
         MaxPool2d-5          [-1, 128, 16, 16]               0
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,456
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,456
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
           Conv2d-14          [-1, 256,

In [5]:
optimizer = SGD(params=model.parameters(), lr=0.08, momentum=.9)
EPOCHS = 25
device = 'cuda' if torch.cuda.is_available() else 'cpu'
for e in range(EPOCHS):
    train(model, optimizer=optimizer, train_dataloader=train_dataloader,device=device,epoch=e )
    test(model,test_dataloader=test_dataloader, device=device, epoch=e)